In [1]:
import os
import re
from bs4 import BeautifulSoup
from colour import Color
import spacy

In [2]:
main_folder = 'Delicious'
html_file_path = os.path.join(main_folder, 'index.html')
css_folder_path = os.path.join(main_folder, 'css')
js_folder_path = os.path.join(main_folder, 'js')

In [3]:
keywords = {
    'topic': ['website', 'web page', 'site', 'online platform', 'internet presence', 'blog', 'ecommerce site', 'portfolio'],
    'location': ['in', 'located in', 'at', 'based in', 'headquartered in', 'global', 'regional', 'national', 'local'],
    'colors': ['colors', 'color scheme', 'color palette', 'shades', 'hue', 'primary color', 'secondary color', 'accent color', 'gradient'],
    'hotline': ['hotline', 'contact', 'phone number', 'customer service', 'support line', 'helpline', 'contact number', 'service hotline']
}

In [5]:
def read_css(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def write_css(file_path, content):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def find_colors(css_content):
    hex_color_regex = r'#[0-9a-fA-F]{3,6}'
    rgb_color_regex = r'rgb\s?\(\s?\d{1,3}\s?,\s?\d{1,3}\s?,\s?\d{1,3}\s?\)'
    rgba_color_regex = r'rgba\s?\(\s?\d{1,3}\s?,\s?\d{1,3}\s?,\s?\d{1,3}\s?,\s?\d?\.?\d+\s?\)'
    
    colors = re.findall(f'{hex_color_regex}|{rgb_color_regex}|{rgba_color_regex}', css_content)
    return colors

def rgba_to_hex(rgba):
    match = re.match(r'rgba\s?\(\s?(\d{1,3})\s?,\s?(\d{1,3})\s?,\s?(\d{1,3})\s?,\s?\d?\.?\d+\s?\)', rgba)
    if match:
        r, g, b = map(int, match.groups())
        return '#{:02x}{:02x}{:02x}'.format(r, g, b)
    return rgba

def generate_new_palette(base_color, old_colors):
    base = Color(base_color)
    new_palette = [base]

    for old_color in old_colors[1:]:
        if 'rgba' in old_color:
            old_color = rgba_to_hex(old_color)
        old_c = Color(old_color)
        new_c = Color(base_color)
        new_c.luminance = old_c.luminance
        new_palette.append(new_c)

    return new_palette

def replace_colors(css_content, old_colors, new_colors):
    for old_color, new_color in zip(old_colors, new_colors):
        css_content = re.sub(re.escape(old_color), new_color, css_content)
    return css_content

def get_linked_css_files(html_file_path):
    with open(html_file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')
    css_files = []
    for link_tag in soup.find_all('link', rel='stylesheet'):
        href = link_tag.get('href')
        if href:
            if href.startswith('http://') or href.startswith('https://'):
                print(f"Skipping external CSS file: {href}")
            else:
                css_files.append(href)
    
    return css_files

In [6]:
def modify_html(html_content, details):
    try:
        html_content = re.sub(r'<title>.*?</title>', f"<title>{details.get('topic', 'My Website')}</title>", html_content)
        if re.search(r'<h1>.*?</h1>', html_content):
            html_content = re.sub(r'<h1>.*?</h1>', f"<h1>Welcome to our {details.get('topic', 'website')}</h1>", html_content)
        else:
            html_content = re.sub(r'<h2>.*?</h2>', f"<h2>Welcome to our {details.get('topic', 'website')}</h2>", html_content)
        if 'location' in details:
            location_html = f"<p>Location: {details['location']}</p>"
            html_content = re.sub(r'<!-- location -->', location_html, html_content)
        if 'hotline' in details:
            hotline_html = f"<p>Hotline: {details['hotline']}</p>"
            html_content = re.sub(r'<!-- hotline -->', hotline_html, html_content)
    except Exception as e:
        print(f"Error occurred during HTML modification: {str(e)}")

    return html_content
def collect_feedback(description, extracted_details, user_feedback):
    try:
        print("Please review the extracted details:")
        print(extracted_details)
        corrected_details = {
            'topic': 'coffee website',
            'location': 'Cairo',
            'colors': ['brown', 'cafe'],
            'hotline': '19888'
        }
        updated_details = {**extracted_details, **corrected_details}

        return updated_details

    except Exception as e:
        print(f"Error during feedback collection: {str(e)}")
        return extracted_details 

nlp = spacy.load('en_core_web_sm')
def extract_details_advanced(description):
    details = {}
    try:
        doc = nlp(description)
        for ent in doc.ents:
            if ent.label_ == 'ORG' and 'topic' not in details:
                details['topic'] = ent.text
            elif ent.label_ == 'GPE' and 'location' not in details:
                details['location'] = ent.text
            elif ent.label_ == 'PRODUCT' and 'topic' not in details:
                details['topic'] = ent.text
        for token in doc:
            if token.text.lower() in keywords['colors'] and token.i + 1 < len(doc):
                details['primary_color'] = doc[token.i + 1].text
            elif token.text.lower() in keywords['hotline'] and token.i + 1 < len(doc):
                next_token = doc[token.i + 1]
                if next_token.like_num:
                    details['hotline'] = next_token.text
    except Exception as e:
        print(f"Error occurred during advanced detail extraction: {str(e)}")

    return details

In [14]:
def main():
    try:
        description = "I want a website about selling coffee, located in Cairo, with brown and cafe colors, and a hotline number 19888."
        extracted_details = extract_details_advanced(description)
        user_feedback = None 
        updated_details = collect_feedback(description, extracted_details, user_feedback)
        with open(html_file_path, 'r', encoding='utf-8') as f:
            html_content = f.read()
        modified_html = modify_html(html_content, updated_details)
        with open(html_file_path, 'w', encoding='utf-8') as f:
            f.write(modified_html)
        css_files = get_linked_css_files(html_file_path)
        for css_file in css_files:
            input_css_path = os.path.join(main_folder, css_file)
            output_css_path = os.path.join(main_folder, css_file)  
        
            css_content = read_css(input_css_path)
            
            old_colors = find_colors(css_content)
            
            base_color = "#0000FF"  
            new_palette = generate_new_palette(base_color, old_colors)
            new_colors = [color.hex for color in new_palette]
            
            updated_css_content = replace_colors(css_content, old_colors, new_colors)
            
            write_css(output_css_path, updated_css_content)
        
        print("Integration completed successfully!")

    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Please review the extracted details:
{'location': 'Cairo', 'primary_color': ','}
Skipping external CSS file: https://fonts.googleapis.com/css?family=Satisfy|Bree+Serif|Candal|PT+Sans
Error occurred: 'rgb(255, 255, 255)' is not a recognized color.


In [17]:
import os
from bs4 import BeautifulSoup
import cssutils

user_bg_image = input("Enter the URL of the background image: ")
user_heading = input("Enter the main heading text: ")
user_paragraph = input("Enter the paragraph text: ")


main_folder = 'burger-free'
html_path = os.path.join(main_folder, 'index.html')
css_folder_path = os.path.join(main_folder, 'css')
css_path = os.path.join('css', 'style.css')


def update_html(html_path, user_bg_image, user_heading, user_paragraph):
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    body_tag = soup.find('body')
    if body_tag:
        body_tag['style'] = f"background-image: url('{user_bg_image}');"
    h1_tag = soup.find('h1')
    if h1_tag:
        h1_tag.string = user_heading
    p_tag = soup.find('p')
    if p_tag:
        p_tag.string = user_paragraph
    with open(html_path, 'w', encoding='utf-8') as file:
        file.write(str(soup))
def update_css(css_path, user_color):
    with open(css_path, 'r', encoding='utf-8') as file:
        css_content = file.read()
    parser = cssutils.CSSParser()
    stylesheet = parser.parseString(css_content)
    
    for rule in stylesheet:
        if rule.type == rule.STYLE_RULE:
            if rule.selectorText == 'body':
                rule.style['background-color'] = user_color
            elif rule.selectorText == 'header':
                rule.style['background'] = user_color
    
    with open(css_path, 'w', encoding='utf-8') as file:
        file.write(stylesheet.cssText.decode('utf-8'))

update_html(html_path, user_bg_image, user_heading, user_paragraph)

print("Website updated successfully.")

Website updated successfully.
